In [1]:
import pandas as pd
import datetime
import numpy as np
import csv
import re
import statistics
import dateutil
import os
from datetime import timedelta
try:
    from tkinter.filedialog import askopenfilename
except:
    pass

In [6]:
import nbimporter
from get_schema import make_schema, time_periodic

In [3]:
def make_df(path, schema):
    s = schema

    df = (
    pd.read_csv(path,
                skiprows=s['skiprows'],
                header=s['header'])
         .drop(columns=s['skipcolumns'])
    )
    
    return df

In [4]:
def num_rpd_periods(df, schema):
    if not schema['file_format'] == 'row_per_day':
        return None
    periods = list(df.drop(columns=[schema['date_column']]).columns)
    try:
        periods = [int(period) for period in periods]
    except:
        return None
    return len(set(periods))

In [5]:
def wide_to_long(df, schema):
    if not schema['file_format'] == 'row_per_day':
        return df, schema
    df = df.melt(id_vars = schema['date_column'],
            var_name = 'time',
            value_name = 'reading')
    try:
        df['time'] = df['time'].astype(int)
    except:
        pass
#     schema['skiprows'] = []
#     schema['skipcolumns'] = []    
    schema['time_column'] = 'time'
    schema['file_format'] = 'single_column'
    schema['time_periodic'] = time_periodic(df, [], 'time')
    return df, schema

In [6]:
def periodic_df(df, schema):
    s = schema
    if not all( [
        s['time_column'],
        s['time_periodic'],
        s['date_column']
    ] ):
        return df, s
    
    period = timedelta(hours=24) / s['time_periodic']
    df[s['time_column']] = df[s['time_column']] * period   
    
    s['time_periodic'] = None
        
    return df, s

In [7]:
def convert_times(df, schema):
    s = schema
    
    if not s['time_column']:
        return df, s
    
    if s['time_periodic']:
        return df, s
    
    try:
        timedelta = (
            pd.to_datetime(df[s['time_column']]) 
            - pd.to_datetime('00:00:00') )

        df[s['time_column']] = timedelta
    except:
        pass
    
    return df, s

In [8]:
def convert_dates(df, schema):
    s = schema
    
    def try_parse_date(col):
        chars = [' ', '-', '_', '/', '#']
        for char in chars:
            dfc = df.copy()
            dfc[col] = dfc[col].str.split(char).str.join(' ')
            try:
                dfc[col] = pd.to_datetime(dfc[col])
                return dfc
            except:
                pass
        return df
        
        
    
    if s['datetime_column']:
        df = try_parse_date(s['datetime_column'])
        
    elif s['date_column'] and s['time_column']:
        df = try_parse_date(s['date_column'])

        df['datetime'] = (
            df[s['date_column']] + df[s['time_column']]
        )
        s['datetime_column'] = 'datetime'
        
    cols_to_drop = []
    if s['date_column']:
        cols_to_drop.append(s['date_column'])
    if s['time_column']:
        cols_to_drop.append(s['time_column'])
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)

    s['date_column'] = None
    s['time_column'] = None
    df = df.set_index(s['datetime_column'])  
    
    df = df.sort_index()

    return df, s

In [9]:
def sum_phases(df, schema):
    s = schema
    rv = df.sum(axis=1).to_frame(name=s['units'])
    return rv

In [10]:
def apply_schema(path):
    s = make_schema(path)
    df = make_df(path, s)
    df, s = wide_to_long(df, s)
    df, s = periodic_df(df, s)
    df, s = convert_times(df, s)
    df, s = convert_dates(df, s)
    df = sum_phases(df, s)
    return df, s

## Tests

In [15]:
path = askopenfilename(initialdir='../..')
path

'/Users/robertbrown/orxa/solarpv/test_input_files/row_per_day_periods.csv'

In [16]:
df, schema = apply_schema(path)

In [19]:
schema

{'skiprows': [0, 1],
 'skipcolumns': ['Total'],
 'date_column': None,
 'datetime_column': 'datetime',
 'file_format': 'single_column',
 'units': 'kWh',
 'header': 0,
 'time_column': None,
 'time_periodic': None}

In [18]:
df

,kWh
datetime,
2018-01-01 00:30:00,62.0
2018-01-01 01:00:00,79.8
2018-01-01 01:30:00,68.5
2018-01-01 02:00:00,50.2
2018-01-01 02:30:00,48.4
...,...
2018-12-31 22:00:00,33.6
2018-12-31 22:30:00,90.7
2018-12-31 23:00:00,80.1
